In [1]:
!pip install neo4j

In [2]:
from langchain.graphs import Neo4jGraph

url = "neo4j+s://xxxx.databases.neo4j.io"
username ="neo4j"
password = "xxxxx"
graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

In [9]:
import yaml
from langchain.graphs.graph_document import (
    Node,
    Relationship,
    GraphDocument,
)
from langchain.docstore.document import Document
from typing import List
import json

# Open the OpenAPI YAML file
with open("openapi.yaml", "r") as file:
    # Load the YAML file using the PyYAML library
    openapi_data = yaml.safe_load(file)

node_list: List[Node] = []
edge_list: List[Relationship] = []

# 添加端点（Endpoint）作为节点，并添加属性
for path, details in openapi_data["paths"].items():
    for method, operation in details.items():
        operation_id = operation.get("operationId")
        parameters = operation.get("parameters")
        # 添加端点节点及其属性
        node = Node(
            id=operation_id,
            type="Endpoint",
            properties={
                "path": path,
                "method": method.upper(),
                "parameters": json.dumps(parameters),
            },
        )
        node_list.append(node)

# 添加模型（Schema）作为节点，并添加属性
for schema, origin_properties in openapi_data["components"]["schemas"].items():
    properties = origin_properties.get("properties")
    description = origin_properties.get("description", "")
    allOf = origin_properties.get("allOf", [])
    # 添加模型节点及其属性
    node = Node(
        id=schema,
        type="Schema",
        properties={
            "description": description,
            "allOf": json.dumps(allOf),
            "properties": json.dumps(properties),
        },
    )
    node_list.append(node)

# 添加边及其属性
for path, details in openapi_data["paths"].items():
    for method, operation in details.items():
        operation_id = operation.get("operationId")

        responses = operation.get("responses", {})
        for status_code, response in responses.items():
            schema_ref = (
                response.get("content", {})
                .get("application/json", {})
                .get("schema", {})
                .get("$ref")
            )
            if schema_ref:
                schema = schema_ref.split("/")[-1]
                # 查找源节点和目标节点
                source_node = next((node for node in node_list if node.id == operation_id), None)
                target_node = next((node for node in node_list if node.id == schema), None)
                if source_node and target_node:
                    # 添加边及其属性
                    edge = Relationship(
                        source=source_node,
                        target=target_node,
                        type=f"Response_{status_code}",
                        properties={},
                    )
                    edge_list.append(edge)

In [10]:
graph_document = GraphDocument(
    nodes = node_list,
    relationships = edge_list,
    source = Document(page_content=str(openapi_data), metadata={})
)
# Store information into a graph
graph.add_graph_documents([graph_document])

In [16]:
from langchain.chains import GraphCypherQAChain

from langchain.chat_models import AzureChatOpenAI

graph.refresh_schema()

In [19]:
cypher_chain = GraphCypherQAChain.from_llm(
    graph=graph,
    cypher_llm=AzureChatOpenAI(temperature=0, deployment_name="gpt-4"),
    qa_llm=AzureChatOpenAI(temperature=0, deployment_name="gpt-35-turbo-16k"),
    validate_cypher=True, # Validate relationship directions
    verbose=True
)

In [20]:
cypher_chain.run("我怎么获取表格中的记录？")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (e:Endpoint)-[r:RESPONSE_200]->(s:Schema) RETURN e, r, s
Full Context:
[{'e': {'path': '/fusion/v1/datasheets/{datasheetId}/records', 'method': 'GET', 'id': 'get-records', 'parameters': '[{"name": "datasheetId", "required": true, "in": "path", "example": "dst0Yj5aNeoHldqvf6", "description": "AITable Datasheet ID", "schema": {"type": "string"}}, {"name": "pageSize", "required": false, "in": "query", "example": 100, "description": "How many records are returned per page. By default, 100 records are returned per page. The value range is an integer from 1 to 1000.", "schema": {"default": 100, "type": "number"}}, {"name": "maxRecords", "required": false, "in": "query", "example": 1000, "description": "How many records are returned in total. If maxRecords and pageSize are used at the same time, and the value of maxRecords is less than the total number of records, only the setting of maxRecords will take effect.", "schema": 

'您可以使用GET请求来获取表格中的记录。使用的API路径是/fusion/v1/datasheets/{datasheetId}/records，其中{datasheetId}是AITable数据表的ID。您可以通过提供的参数来指定返回的记录数量、排序方式、筛选条件等。默认情况下，每页返回100条记录。如果需要获取所有记录，可以使用maxRecords参数来指定返回的记录总数。'

In [21]:
cypher_chain.run("我想要获取筛选后的记录，该怎么办？")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (e:Endpoint)-[r:RESPONSE_200]->(s:Schema) RETURN e, r, s LIMIT 10
Full Context:
[{'e': {'path': '/fusion/v1/datasheets/{datasheetId}/records', 'method': 'GET', 'id': 'get-records', 'parameters': '[{"name": "datasheetId", "required": true, "in": "path", "example": "dst0Yj5aNeoHldqvf6", "description": "AITable Datasheet ID", "schema": {"type": "string"}}, {"name": "pageSize", "required": false, "in": "query", "example": 100, "description": "How many records are returned per page. By default, 100 records are returned per page. The value range is an integer from 1 to 1000.", "schema": {"default": 100, "type": "number"}}, {"name": "maxRecords", "required": false, "in": "query", "example": 1000, "description": "How many records are returned in total. If maxRecords and pageSize are used at the same time, and the value of maxRecords is less than the total number of records, only the setting of maxRecords will take effect.", "

'您可以使用`filterByFormula`参数来筛选记录。该参数使用智能公式来过滤记录。您可以参考[公式概述](https://help.aitable.ai/docs/guide/manual-formula-field-overview)来使用公式进行筛选。例如，您可以使用`filterByFormula={Title}="tittle 1"`来筛选出标题为"tittle 1"的记录。请注意，您需要使用`encodeURIComponent()`函数来转义公式中的特殊字符。'

In [23]:
cypher_chain.run("报错了，401，咋办")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
Sorry, but I can't assist with that.


ValueError: Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'Sorry': expected
  "ALTER"
  "CALL"
  "CREATE"
  "DEALLOCATE"
  "DELETE"
  "DENY"
  "DETACH"
  "DROP"
  "DRYRUN"
  "ENABLE"
  "FOREACH"
  "GRANT"
  "LOAD"
  "MATCH"
  "MERGE"
  "OPTIONAL"
  "REALLOCATE"
  "REMOVE"
  "RENAME"
  "RETURN"
  "REVOKE"
  "SET"
  "SHOW"
  "START"
  "STOP"
  "TERMINATE"
  "UNWIND"
  "USE"
  "USING"
  "WITH" (line 1, column 1 (offset: 0))
"Sorry, but I can't assist with that."
 ^}